## Collecting song information

In [1]:
!pip install spotipy

In [2]:
import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [3]:
cid = "3519692942004325a2c7160c90717ca5"
secret = "9243be1df96e48bb829c4b07254bd82c"
redirect_uri = 'https://github.com/manav-s/audioalchemy'
scope = 'playlist-modify-public playlist-modify-private'

In [4]:
# client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

auth_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
playlist_link = "https://open.spotify.com/playlist/1hH8ZCf9orRRymgRP95jXb?si=e45b5924a9c0435b&pt=e744ed4e7acc7c495604af2b8214e392"
uri = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(uri)["items"]]

In [30]:
print(sp.audio_features(track_uris)[0])

{'danceability': 0.841, 'energy': 0.733, 'key': 1, 'loudness': -5.077, 'mode': 1, 'speechiness': 0.142, 'acousticness': 0.0246, 'instrumentalness': 0, 'liveness': 0.174, 'valence': 0.284, 'tempo': 130.009, 'type': 'audio_features', 'id': '7xVLFuuYdAvcTfcP3IG3dS', 'uri': 'spotify:track:7xVLFuuYdAvcTfcP3IG3dS', 'track_href': 'https://api.spotify.com/v1/tracks/7xVLFuuYdAvcTfcP3IG3dS', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7xVLFuuYdAvcTfcP3IG3dS', 'duration_ms': 283693, 'time_signature': 4}


In [22]:
import pandas as pd

# Initialize empty lists for columns
song_names = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

# Loop through the tracks and append the data to the respective lists
for track in sp.playlist_tracks(playlist_link)["items"]:
    track_name = track["track"]["name"]
    song_names.append(track_name)
    
    audio_features = sp.audio_features(track["track"]["uri"])[0]
    danceability.append(audio_features["danceability"])
    energy.append(audio_features["energy"])
    key.append(audio_features["key"])
    loudness.append(audio_features["loudness"])
    mode.append(audio_features["mode"])
    speechiness.append(audio_features["speechiness"])
    acousticness.append(audio_features["acousticness"])
    instrumentalness.append(audio_features["instrumentalness"])
    liveness.append(audio_features["liveness"])
    valence.append(audio_features["valence"])
    tempo.append(audio_features["tempo"])

# Create a DataFrame
data = {
    "Song Name": song_names,
    "Danceability": danceability,
    "Energy": energy,
    "Key": key,
    "Loudness": loudness,
    "Mode": mode,
    "Speechiness": speechiness,
    "Acousticness": acousticness,
    "Instrumentalness": instrumentalness,
    "Liveness": liveness,
    "Valence": valence,
    "Tempo": tempo
}

df = pd.DataFrame(data)
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,A Tale of 2 Citiez,0.559,0.556,6,-7.844,1,0.531,0.553000,0.000000,0.2510,0.362,189.861
1,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103
2,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793
3,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983
4,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013
96,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827
97,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239
98,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080


# Score

We need a weighting system that identifies the importance of each feature.

In [66]:
def get_relative_key(key, mode):
    if mode == 1:  # Major key
        return (key + 9) % 12  # Relative minor key
    else:  # Minor key
        return (key + 3) % 12  # Relative major key

def key_compatibility(key1, mode1, key2, mode2):
    if key1 == key2:
        return True
    if mode1 != mode2 and (key1 == get_relative_key(key2, mode2) or key2 == get_relative_key(key1, mode1)):
        return True
    return False

def genre_similarity(genres1, genres2):
    if not genres1 or not genres2:
        return 0
    shared_genres = len(set(genres1).intersection(genres2))
    return shared_genres / max(len(genres1), len(genres2))

# The closer the score to 0, the better the transition 
def evaluate_transition(song1, song2):
    score = 0
    
    # Weights for different attributes
    weights = {
        'danceability': 7,
        'energy': 5,
        'loudness': 1,
        'tempo': 20,
        'valence': 5,
        'genre': 8 
    }

    # Check key compatibility
    if not key_compatibility(song1['key'], song1['mode'], song2['key'], song2['mode']):
        score += 25  # Return a large score if keys are not compatible

    # Calculate the differences in attributes
    diff = {}
    for attribute in weights:
        if attribute == 'genre':
            diff[attribute] = 1 - genre_similarity(song1[attribute], song2[attribute])
        else:
            diff[attribute] = abs(song1[attribute] - song2[attribute])

    # Normalize loudness differences
    diff['loudness'] /= 60  # Assume max difference is 60 dB

    # Calculate tempo difference considering double/half time mixing
    tempo_diff = min(
        abs(song1['tempo'] - song2['tempo']),
        abs(song1['tempo'] * 2 - song2['tempo']),
        abs(song1['tempo'] / 2 - song2['tempo'])
    )
    diff['tempo'] = tempo_diff / 200  # Normalize tempo difference

    # Calculate the transition score
    for attribute in weights:
        print("attribute: ", attribute, "   score: ", weights[attribute])
        score += weights[attribute] * diff[attribute]

    return score


### Transition Score Testing

In [52]:
def print_song_data(song):
    print("Song Name:", song["track_name"])
    print("Danceability:", song["danceability"])
    print("Energy:", song["energy"])
    print("Key:", song["key"])
    print("Loudness:", song["loudness"])
    print("Mode:", song["mode"])
    print("Valence:", song["valence"])
    print("Tempo:", song["tempo"])
    print("Genre:", ', '.join(song["genre"]))
    print()

def get_related_artist_genres(artist_id):
    related_artists = sp.artist_related_artists(artist_id)
    genres = []
    for artist in related_artists['artists']:
        genres.extend(artist['genres'])
    return list(set(genres))

def get_song_data(track_id):
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)[0]
    artist_id = track['artists'][0]['id']
    genres = get_related_artist_genres(artist_id)

    song_data = audio_features.copy()
    song_data['genre'] = genres
    song_data['track_name'] = track['name']
    return song_data

#### Same Song score

In [67]:
song1_id = "0hURIUSiPFIv7dzlejdf3N"
song2_id = "0hURIUSiPFIv7dzlejdf3N"

song1 = get_song_data(song1_id)
song2 = get_song_data(song2_id)

print("Song 1 Info:")
print_song_data(song1)

print("Song 2 Info:")
print_song_data(song2)

transition_score = evaluate_transition(song1, song2)
print(f"Transition score between the two songs: {transition_score:.2f}")

Song 1 Info:
Song Name: The Sweetest Taboo - Quiroga Remix
Danceability: 0.802
Energy: 0.898
Key: 6
Loudness: -9.163
Mode: 1
Valence: 0.628
Tempo: 128.021
Genre: batak, trio batak, classic indo pop, lagu maluku, indonesian worship, indonesian city pop

Song 2 Info:
Song Name: The Sweetest Taboo - Quiroga Remix
Danceability: 0.802
Energy: 0.898
Key: 6
Loudness: -9.163
Mode: 1
Valence: 0.628
Tempo: 128.021
Genre: batak, trio batak, classic indo pop, lagu maluku, indonesian worship, indonesian city pop

attribute:  danceability    score:  7
attribute:  energy    score:  5
attribute:  loudness    score:  1
attribute:  tempo    score:  20
attribute:  valence    score:  5
attribute:  genre    score:  8
Transition score between the two songs: 0.00


#### Good Song score

In [68]:
song1_id = "0yLdNVWF3Srea0uzk55zFn"
song2_id = "3RfNQMIeuL2QC9l4VxOMoj"

song1 = get_song_data(song1_id)
song2 = get_song_data(song2_id)

print("Song 1 Info:")
print_song_data(song1)

print("Song 2 Info:")
print_song_data(song2)

transition_score = evaluate_transition(song1, song2)
print(f"Transition score between the two songs: {transition_score:.2f}")

Song 1 Info:
Song Name: Flowers
Danceability: 0.707
Energy: 0.681
Key: 0
Loudness: -4.325
Mode: 1
Valence: 0.646
Tempo: 117.999
Genre: post-teen pop, uk pop, swedish pop, boy band, swedish electropop, electropop, dance pop, swedish synthpop, candy pop, australian hip hop, art pop, metropopolis, transpop, talent show, girl group, alt z, pop

Song 2 Info:
Song Name: Dream
Danceability: 0.684
Energy: 0.675
Key: 0
Loudness: -5.886
Mode: 1
Valence: 0.846
Tempo: 117.97
Genre: post-teen pop, uk pop, swedish pop, boy band, swedish electropop, electropop, dance pop, swedish synthpop, candy pop, australian hip hop, art pop, metropopolis, transpop, talent show, girl group, alt z, pop

attribute:  danceability    score:  7
attribute:  energy    score:  5
attribute:  loudness    score:  1
attribute:  tempo    score:  20
attribute:  valence    score:  5
attribute:  genre    score:  8
Transition score between the two songs: 1.22


#### Medium Song score

In [69]:
song1_id = "1j6kDJttn6wbVyMaM42Nxm"
song2_id = "2I9foKseoFQh07p6sD2voE"

song1 = get_song_data(song1_id)
song2 = get_song_data(song2_id)

print("Song 1 Info:")
print_song_data(song1)

print("Song 2 Info:")
print_song_data(song2)

transition_score = evaluate_transition(song1, song2)
print(f"Transition score between the two songs: {transition_score:.2f}")

Song 1 Info:
Song Name: Lord Pretty Flacko Jodye 2 (LPFJ2)
Danceability: 0.485
Energy: 0.72
Key: 6
Loudness: -5.991
Mode: 1
Valence: 0.0471
Tempo: 207.982
Genre: detroit hip hop, pop rap, trap, conscious hip hop, boom bap, atl hip hop, chicago rap, dark trap, virginia hip hop, hip hop, tennessee hip hop, queens hip hop, indiana hip hop, chicago bop, experimental hip hop, gangster rap, new orleans rap, east coast hip hop, rap, southern hip hop, drill, escape room, nyc rap, crunk, miami hip hop, dirty south rap, memphis hip hop, chicago drill, underground hip hop, alternative hip hop, psychedelic hip hop

Song 2 Info:
Song Name: Many Men (Wish Death)
Danceability: 0.653
Energy: 0.826
Key: 6
Loudness: -2.93
Mode: 0
Valence: 0.416
Tempo: 108.52
Genre: detroit hip hop, pop rap, singeli, r&b, trap, west coast rap, atl hip hop, hip hop, queens hip hop, hardcore hip hop, bronx hip hop, gangster rap, hip pop, g funk, east coast hip hop, rap, southern hip hop, nyc rap, crunk, urban contemporary,

#### Bad Song score

In [70]:
song1_id = "1j6kDJttn6wbVyMaM42Nxm"
song2_id = "0hURIUSiPFIv7dzlejdf3N"

song1 = get_song_data(song1_id)
song2 = get_song_data(song2_id)

print("Song 1 Info:")
print_song_data(song1)

print("Song 2 Info:")
print_song_data(song2)

transition_score = evaluate_transition(song1, song2)
print(f"Transition score between the two songs: {transition_score:.2f}")

Song 1 Info:
Song Name: Lord Pretty Flacko Jodye 2 (LPFJ2)
Danceability: 0.485
Energy: 0.72
Key: 6
Loudness: -5.991
Mode: 1
Valence: 0.0471
Tempo: 207.982
Genre: detroit hip hop, pop rap, trap, conscious hip hop, boom bap, atl hip hop, chicago rap, dark trap, virginia hip hop, hip hop, tennessee hip hop, queens hip hop, indiana hip hop, chicago bop, experimental hip hop, gangster rap, new orleans rap, east coast hip hop, rap, southern hip hop, drill, escape room, nyc rap, crunk, miami hip hop, dirty south rap, memphis hip hop, chicago drill, underground hip hop, alternative hip hop, psychedelic hip hop

Song 2 Info:
Song Name: The Sweetest Taboo - Quiroga Remix
Danceability: 0.802
Energy: 0.898
Key: 6
Loudness: -9.163
Mode: 1
Valence: 0.628
Tempo: 128.021
Genre: batak, trio batak, classic indo pop, lagu maluku, indonesian worship, indonesian city pop

attribute:  danceability    score:  7
attribute:  energy    score:  5
attribute:  loudness    score:  1
attribute:  tempo    score:  20


## Implementing Beam Search

In [24]:
# Loop through the tracks and print the track name and its artist's genres
for item in sp.playlist_tracks(uri)["items"]:
    track = item["track"]
    track_name = track["name"]
    artist_id = track["artists"][0]["id"]

    # Get the artist's genres
    artist = sp.artist(artist_id)
    genres = artist["genres"]

    print(f"Track: {track_name}\nArtist's Genres: {', '.join(genres)}\n")


Track: A Tale of 2 Citiez
Artist's Genres: conscious hip hop, hip hop, north carolina hip hop, rap

Track: Be Something
Artist's Genres: chicago rap, rap

Track: New Memories (Intro)
Artist's Genres: desi hip hop, desi pop, desi trap, punjabi hip hop, punjabi pop

Track: Having Our Way (feat. Drake)
Artist's Genres: atl hip hop, hip hop, rap, trap

Track: EVERY CHANCE I GET (feat. Lil Baby & Lil Durk)
Artist's Genres: hip hop, miami hip hop, pop rap, rap

Track: Light It Up (feat. Pop Smoke)
Artist's Genres: atl hip hop, hip hop, rap, trap

Track: 90210 (feat. Kacy Hill)
Artist's Genres: hip hop, rap, slap house

Track: Stronger
Artist's Genres: chicago rap, hip hop, rap

Track: Juicy
Artist's Genres: east coast hip hop, gangster rap, hardcore hip hop, hip hop, rap

Track: Ambitionz Az A Ridah
Artist's Genres: g funk, gangster rap, hip hop, rap, west coast rap

Track: 1942 Flows
Artist's Genres: hip hop, philly rap, rap, southern hip hop, trap

Track: Solid (feat. Drake)
Artist's Genre

## Reordering the Spotify playlist

We will add the new index of each song to the dataframe

In [20]:
df['new_index'] = pd.Series(None, index=df.index)
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,new_index
0,A Tale of 2 Citiez,0.559,0.556,6,-7.844,1,0.531,0.553000,0.000000,0.2510,0.362,189.861,NaN
1,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103,NaN
2,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793,NaN
3,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983,NaN
4,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013,NaN
96,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827,NaN
97,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239,NaN
98,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080,NaN


In [10]:
# testing by reversing the index
df['new_index'] = df.index[::-1]
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,new_index
0,"Work REMIX (feat. A$AP Rocky, French Montana, ...",0.841,0.733,1,-5.077,1,0.142,0.024600,0.000000,0.1740,0.284,130.009,99
1,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080,98
2,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239,97
3,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827,96
4,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062,4
96,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983,3
97,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793,2
98,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103,1


In [11]:
def reorder_playlist(playlist_id, original_index, new_index):
    # Get the current tracks in the playlist
    current_tracks = sp.playlist_tracks(playlist_id)['items']
    current_uris = [track['track']['uri'] for track in current_tracks]
    
    # Rearrange the tracks according to the new index
    new_uris = [current_uris[idx] for idx in new_index]
    
    # Replace the tracks in the playlist with the new order
    sp.playlist_replace_items(playlist_id, new_uris)
    
    # Update the DataFrame with the new indices
    new_order = [i for i in range(len(original_index))]
    df = original_index.to_frame()
    df['new_index'] = new_order

In [12]:
reorder_playlist(uri, df.index, df['new_index'])

In [13]:
# Initialize empty lists for columns
song_names = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

# Loop through the tracks and append the data to the respective lists
for track in sp.playlist_tracks(uri)["items"]:
    track_name = track["track"]["name"]
    song_names.append(track_name)
    
    audio_features = sp.audio_features(track["track"]["uri"])[0]
    danceability.append(audio_features["danceability"])
    energy.append(audio_features["energy"])
    key.append(audio_features["key"])
    loudness.append(audio_features["loudness"])
    mode.append(audio_features["mode"])
    speechiness.append(audio_features["speechiness"])
    acousticness.append(audio_features["acousticness"])
    instrumentalness.append(audio_features["instrumentalness"])
    liveness.append(audio_features["liveness"])
    valence.append(audio_features["valence"])
    tempo.append(audio_features["tempo"])

# Create a DataFrame
data = {
    "Song Name": song_names,
    "Danceability": danceability,
    "Energy": energy,
    "Key": key,
    "Loudness": loudness,
    "Mode": mode,
    "Speechiness": speechiness,
    "Acousticness": acousticness,
    "Instrumentalness": instrumentalness,
    "Liveness": liveness,
    "Valence": valence,
    "Tempo": tempo
}

df = pd.DataFrame(data)
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,A Tale of 2 Citiez,0.559,0.556,6,-7.844,1,0.531,0.553000,0.000000,0.2510,0.362,189.861
1,Be Something,0.829,0.520,8,-10.340,1,0.294,0.023200,0.000000,0.1340,0.248,136.103
2,New Memories (Intro),0.708,0.692,8,-7.567,1,0.211,0.011700,0.000000,0.1130,0.141,95.793
3,Having Our Way (feat. Drake),0.927,0.538,1,-3.917,1,0.116,0.031200,0.000000,0.0945,0.221,125.983
4,EVERY CHANCE I GET (feat. Lil Baby & Lil Durk),0.477,0.727,2,-4.053,1,0.286,0.000866,0.000000,0.1080,0.227,150.062
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Welcome To The Party,0.637,0.723,2,-5.547,1,0.091,0.013400,0.000079,0.1960,0.228,143.013
96,Global,0.672,0.565,1,-6.504,1,0.441,0.106000,0.000000,0.1130,0.313,163.827
97,Ali Bomaye,0.381,0.690,10,-5.312,0,0.455,0.020900,0.000000,0.4380,0.223,78.239
98,New Level (feat. Future),0.734,0.703,1,-1.690,0,0.132,0.034400,0.000000,0.1620,0.152,121.080
